### Comparing loss/validation curves and final scores across experiments

TODO: 
   - test multi-task setups appear correctly (not urgent ...) 

In [1]:
import pandas as pds
import os.path
from glob import glob
import altair as alt
import json
%matplotlib inline

from collect_scores import config, tasks as seg_tasks, pdtb_tasks, models, collect_final_result

/home/muller/miniconda3/envs/jiant3.8/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [19]:
RUN_DIR = "../runs"

config["exp_dir"] = RUN_DIR
config["tasks"] = seg_tasks | pdtb_tasks
TEST_TASK = "deu_rst_pcc"



In [20]:
val_scores = collect_final_result(config["tasks"],["roberta","roberta-large"],config,best_metadata=False)

In [21]:
val_scores

,task,task_type,precision,recall,f1,freeze_layers,setup,model
0,fas_rst_prstc,conllu,0.919521,0.932292,0.925862,,single,roberta
0,fra_sdrt_annodis,conllu,0.893694,0.892086,0.892889,,single,roberta
0,ita_pdtb_luna,conllu,0.902913,0.669065,0.768595,,single,roberta
0,rus_rst_rrt,conllu,0.854773,0.876455,0.865478,,single,roberta
0,eus_rst_ert,conllu,0.903561,0.899557,0.901554,,single,roberta
0,eng_sdrt_stac,conllu,0.943493,0.954939,0.949182,,single,roberta
0,zho_rst_gcdt,conllu,0.888794,0.920283,0.904265,,single,roberta
0,eng_dep_scidtb,conllu,0.940400,0.970423,0.955176,,single,roberta
0,nld_rst_nldt,conllu,0.967647,0.959184,0.963397,,single,roberta
0,nld_rst_nldt,conllu,0.970760,0.967930,0.969343,,single,roberta


In [16]:

TEST_TASK = "deu_rst_pcc"

test_dir = retrieve_expe_dir(TEST_TASK,"conllu","roberta",config)
print(test_dir)
print(get_taskname_from_dir(test_dir))
path = get_last_run(test_dir)
log = get_log(path,"metrics")
df = collect_expe_results(["deu_rst_pcc"],"conllu",["roberta","roberta-large"],config)

# Example visualizations of loss/validation curves
#alt.data_transformers.disable_max_rows()
filter = (df["task"]=="deu.rst.pcc") & (df["model"]=="roberta-large") #& (df["global_step"]<5000)
df[filter]["f1"].plot()

In [17]:
# doesnt show ... why ? interpreter problem ?
# alt.renderers.enable('default')

raw_loss = alt.Chart(df[filter]).mark_line().encode(
  x='task_step',# or global_step
  y='loss_val',
  color='model:N' # setup or model
)

raw_f1 = alt.Chart(df[filter & (df["f1"].notnull())]).mark_point().encode(
  x='task_step',# global_step
  y='f1',
  color='model:N'
)

f1_labels = raw_f1.mark_text(
    align="center",
    dy= -20
).encode(text=alt.Text("f1:Q",format=".1%"))

# rolling average 
avg_loss = alt.Chart(df[filter]).mark_line(color="green"
).transform_window(
    rolling_mean='mean(loss_val)',
    frame=[-10,0]
).encode(
    x='task_step',# global_step
    y='rolling_mean:Q',
    color="model:N"
    #color="task:N"
)

#alt.layer(avg_loss + raw_f1).interactive() #+ f1_labels
avg_loss | (raw_f1 + f1_labels)

NameError: name 'df' is not defined

In [18]:

filter = val_scores["task"].isin(pdtb_tasks)
print("===== Scores pdtb ========")
print("---mean=",val_scores[filter].groupby("model").mean()["f1"])
print("---std dev=",val_scores[filter].groupby("model").std()["f1"])
filter = ~(val_scores["task"].isin(pdtb_tasks))
print("===== Scores segmentation ========")
print("---mean=",val_scores[filter].groupby("model").mean()["f1"])
print("---std dev=",val_scores[filter].groupby("model").std()["f1"])

===== Scores pdtb ========
---mean= model
roberta    0.768595
Name: f1, dtype: float64
---std dev= model
roberta   NaN
Name: f1, dtype: float64
===== Scores segmentation ========
---mean= model
roberta          0.917076
roberta-large    0.890646
Name: f1, dtype: float64
---std dev= model
roberta          0.051306
roberta-large    0.057228
Name: f1, dtype: float64


/tmp/ipykernel_265737/1181446257.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("---mean=",val_scores[filter].groupby("model").mean()["f1"])
/tmp/ipykernel_265737/1181446257.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("---std dev=",val_scores[filter].groupby("model").std()["f1"])
/tmp/ipykernel_265737/1181446257.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  print("---mean=",val_scores[fil

In [84]:
# to compare multiple models across multiple datasets + CI
# better if averaged on multiple runs
# inspired from: https://altair-viz.github.io/gallery/grouped_bar_chart_with_error_bars.html
scores = alt.Chart().mark_bar().encode(
    x="model",
    y="f1",
    color="model"
)

error_bars = alt.Chart().mark_errorbar(extent='ci').encode(
    x='model',
    y='f1'
)
f1_labels = scores.mark_text(
    align="left",
    dy= -20, dx=-10
).encode(text=alt.Text("f1:Q",format=".1%"))


alt.layer(scores, error_bars, f1_labels,data=val_scores).facet(
    column='task:N'
).interactive()

/home/muller/miniconda3/envs/jiant3.8/lib/python3.8/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.FacetChart(...)